In [1]:
import os
import cv2
import numpy as np
import shutil
from deepface import DeepFace
import threading
import time
import queue

## Tools & tests

Creating samples folder (celebrities)

In [9]:
data_dir = 'Datasets/famosos'

for directory in os.listdir(data_dir):
    first_file = os.listdir(os.path.join(data_dir, directory))[0]
    shutil.copyfile(os.path.join(data_dir, directory, first_file), os.path.join("Samples", f"{directory}.jpg"))

Lookalike test

In [13]:
result = DeepFace.verify("person1.jpg", f"Samples/Angelina Jolie.jpg")

In [14]:
result

{'verified': False,
 'distance': 0.7698307337090413,
 'threshold': 0.4,
 'model': 'VGG-Face',
 'detector_backend': 'opencv',
 'similarity_metric': 'cosine',
 'facial_areas': {'img1': {'x': 208, 'y': 37, 'w': 120, 'h': 120},
  'img2': {'x': 74, 'y': 115, 'w': 287, 'h': 287}},
 'time': 0.54}

Age, render & race test

In [5]:
emotionImg = cv2.imread('./person1.jpg')
obj = DeepFace.analyze(emotionImg, enforce_detection=False, actions=['age', 'gender', 'race', 'emotion'])

print(obj)

Action: emotion: 100%|██████████| 4/4 [00:00<00:00,  4.77it/s]

[{'age': 29, 'region': {'x': 208, 'y': 37, 'w': 120, 'h': 120}, 'gender': {'Woman': 4.4589015146812017e-05, 'Man': 99.99995231628418}, 'dominant_gender': 'Man', 'race': {'asian': 0.32646742183715105, 'indian': 7.5354211032390594, 'black': 0.32972635235637426, 'white': 32.79293179512024, 'middle eastern': 32.33891725540161, 'latino hispanic': 26.676541566848755}, 'dominant_race': 'white', 'emotion': {'angry': 3.388790208301382e-08, 'disgust': 5.378273185664511e-13, 'fear': 9.986144302869349e-06, 'happy': 96.66207989002227, 'sad': 2.6955637382574697e-06, 'surprise': 0.008525826995080203, 'neutral': 3.329385841415625}, 'dominant_emotion': 'happy'}]


## Lookalike search with celebrities (images)

In [22]:
smallest_distance = None

for file in os.listdir("Samples"):
    if file.endswith(".jpg"):
        # result = DeepFace.verify("./Images/person1.jpg", f"Samples/{file}")
        # result = DeepFace.verify("./Images/person2.jpg", f"Samples/{file}")
        result = DeepFace.verify("./Images/match.jpg", f"Samples/{file}")
        if result['verified']:
            print("This person looks exactly like", file.split(".")[0])
            break
        if smallest_distance is None: # First instance
            smallest_distance = (file.split(".")[0], result['distance'])
        else:
            smallest_distance = (file.split(".")[0], result['distance']) if result['distance'] < smallest_distance[1] else smallest_distance
else:
    print(f"No exact match found! Closest match is {smallest_distance[0]}")

No exact match found! Closest match is Hugh Jackman


## Lookalike search with celebrities (video)

In [2]:
# Ruta al directorio que contiene las carpetas de categorías (personas)
carpeta_referencia = "./Samples"

# Obtener la lista de categorías (nombres de personas)
archivos_referencia = os.listdir(carpeta_referencia)

# Inicializar variables para el resultado final
persona_mas_parecida = None
distancia_minima = float('inf')

# Inicializar la captura de la cámara
cap = cv2.VideoCapture(0)

# Capturar el fotograma de la cámara
ret, frame = cap.read()
frame = cv2.resize(frame, (640, 480))

# Guardar la imagen temporal
cv2.imwrite("imagen_temporal.jpg", frame)
imagen_a_comparar = "imagen_temporal.jpg"

# Analizar la imagen para obtener información de edad, género y emoción
info_comparar = DeepFace.analyze(imagen_a_comparar, enforce_detection=False, actions=['age', 'gender', 'emotion'])[0]
edad_comparar = info_comparar['age']
genero_comparar = info_comparar['dominant_gender']
emocion_comparar = info_comparar['dominant_emotion']

# Iterar sobre cada categoría (persona) en el repositorio
# Iterar sobre cada archivo en la carpeta de referencia
for archivo_referencia in archivos_referencia:
    # Ruta completa a la imagen de referencia de la persona
    imagen_referencia = os.path.join(carpeta_referencia, archivo_referencia)
    imagen_referencia = imagen_referencia.replace("\\", "/")

    # Analizar la imagen de referencia para obtener información de edad, género y emoción
    info_referencia = DeepFace.analyze(imagen_referencia, enforce_detection=False, actions=['age', 'gender', 'emotion'])[0]
    edad_referencia = info_referencia['age']
    genero_referencia = info_referencia['dominant_gender']

    # Calcular la diferencia en edad y género (otorgando pesos según la importancia)
    diff_edad = abs(edad_comparar - edad_referencia)
    peso_edad = 0.5
    diff_genero = 0 if genero_comparar.lower() == genero_referencia.lower() else 1
    peso_genero = 0.5

    # Realizar la comparación de similitud
    result = DeepFace.verify(imagen_a_comparar, imagen_referencia, model_name='Facenet', distance_metric='euclidean_l2', enforce_detection=False)
    distancia_facial = result["distance"]

    # Calcular la similitud total basada en la diferencia de edad, género y la distancia facial
    similitud = peso_edad * diff_edad + distancia_facial # + peso_genero * diff_genero

    # Actualizar si encontramos una similitud menor
    if similitud < distancia_minima:
        distancia_minima = similitud
        persona_mas_parecida = os.path.splitext(archivo_referencia)[0]  # Eliminar la extensión para obtener la categoría

# Cargar la imagen del actor/actriz más parecido
imagen_act_referencia = cv2.imread(os.path.join(carpeta_referencia, f"{persona_mas_parecida}.jpg"))

# Mostrar la imagen en la parte superior del fotograma
if imagen_act_referencia is not None:
    cv2.imshow('Actor/Actriz Más Parecido(a)', imagen_act_referencia)

# Liberar la captura de la cámara
cap.release()

# Mostrar el resultado en el fotograma
# cv2.putText(frame, f"Persona mas parecida: {persona_mas_parecida}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
# cv2.putText(frame, f"Similitud total: {distancia_minima}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
# cv2.putText(frame, f"Edad: {edad_comparar}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
# cv2.putText(frame, f"Emoción: {emocion_comparar}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
cv2.imshow('Comparación de caras', frame)

# Imprimir el resultado
print(f"La persona más parecida es {persona_mas_parecida} con una similitud total de {distancia_minima}")

# Eliminar la imagen temporal al finalizar
os.remove("imagen_temporal.jpg")

# Esperar hasta que se presione una tecla y cerrar todas las ventanas
cv2.waitKey(0)
cv2.destroyAllWindows()


Action: emotion: 100%|██████████| 3/3 [00:00<00:00,  5.82it/s]


La persona más parecida es Sandra Bullock con una similitud total de 1.316648349942769


## Lookalike search with memes (video)

In [3]:
# Ruta al directorio que contiene las carpetas de categorías (personas)
carpeta_referencia = "./Datasets/memes/"

imagen_por_defecto_path = './Datasets/memes/default.png'


# Obtener la lista de categorías (nombres de personas)
archivos_referencia = os.listdir(carpeta_referencia)

# Tamaño de la imagen después de redimensionar
resize_width = 320
resize_height = 240

# Inicializar variables para el resultado final
persona_mas_parecida = None
distancia_minima = float('inf')
img_persona_parecida = None

# Inicializar la captura de la cámara
cap = cv2.VideoCapture(0)

# Lock para asegurar acceso seguro a las variables compartidas
frame_lock = threading.Lock()
result_lock = threading.Lock()

frame_queue = queue.Queue(maxsize=1)
result_info_queue = queue.Queue(maxsize=1)

# Variable de evento para notificar a los hilos que deben finalizar
exit_event = threading.Event()

# Variables compartidas entre hilos
frame = None
result_info = None

def read_camera():
    global frame, persona_mas_parecida, persona_anterior, index_referencia
    persona_mas_parecida = None  # Inicializa persona_mas_parecida antes del bucle
    img_persona_parecida = None 

    while not exit_event.is_set():
        ret, current_frame = cap.read()
        if ret and current_frame is not None:
            with frame_lock:
                frame = cv2.resize(current_frame, (resize_width, resize_height))
            try:
                frame_queue.put_nowait(frame)
            except queue.Full:
                pass
            # Display the real-time camera frame
            cv2.imshow('Camera Frame', frame)
        else:
            print("Error: Failed to capture frame from the camera.")

        # Visualizar el texto en la imagen principal
        with result_lock:
            try:
                if persona_mas_parecida is not None:
                    # Si la persona más parecida ha cambiado desde la iteración anterior, cargar la nueva imagen
                    if index_referencia == 0:
                        img_persona_parecida_path = os.path.join(carpeta_referencia, f"{persona_mas_parecida}.jpg").replace("\\", "/")
                        img_persona_parecida = cv2.resize(cv2.imread(img_persona_parecida_path), (resize_width, resize_height))
                    if img_persona_parecida is not None:
                        cv2.imshow('Persona más parecida', img_persona_parecida)
                else:
                    img_por_defecto = cv2.resize(cv2.imread(imagen_por_defecto_path), (resize_width, resize_height))
                    cv2.imshow('Persona más parecida', img_por_defecto)
            except Exception as e:
                print(f"Error al mostrar imágenes: {e}")

        # Salir del bucle si se presiona 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

def process_image():
    global persona_mas_parecida, distancia_minima, result_info, index_referencia

    # Inicializar el índice
    index_referencia = 0

    while not exit_event.is_set():
        try:
            frame = frame_queue.get_nowait()
        except queue.Empty:
            time.sleep(0.1)
            continue

        cv2.imwrite("imagen_temporal.jpg", frame)
        imagen_a_comparar = "imagen_temporal.jpg"

        # Redimensionar la imagen a comparar
        img_comparar = cv2.resize(cv2.imread(imagen_a_comparar), (resize_width, resize_height))

        # Analizar la imagen para obtener información de edad, género y emoción
        info_comparar = DeepFace.analyze(img_comparar, enforce_detection=False, actions=['age', 'gender', 'emotion'])[0]
        edad_comparar = info_comparar['age']
        genero_comparar = info_comparar['dominant_gender']

        # Reinicializar variables antes de la comparación
        with result_lock:
            distancia_minima = float('inf')
            persona_mas_parecida = None

        # Iterar sobre cada categoría (persona) en el repositorio
        while index_referencia < len(archivos_referencia):
            archivo_referencia = archivos_referencia[index_referencia]
            imagen_referencia = os.path.join(carpeta_referencia, archivo_referencia).replace("\\", "/")

            # Redimensionar la imagen de referencia
            img_referencia = cv2.resize(cv2.imread(imagen_referencia), (resize_width, resize_height))

            # Analizar la imagen de referencia para obtener información de edad, género y emoción
            info_referencia = DeepFace.analyze(img_referencia, enforce_detection=False, actions=['age', 'gender', 'emotion'])[0]
            edad_referencia = info_referencia['age']
            genero_referencia = info_referencia['dominant_gender']

            # Calcular la diferencia en edad y género (otorgando pesos según la importancia)
            diff_edad = abs(edad_comparar - edad_referencia)
            peso_edad = 0.5
            diff_genero = 0 if genero_comparar.lower() == genero_referencia.lower() else 1
            peso_genero = 0.5

            # Realizar la comparación de similitud
            result = DeepFace.verify(img_comparar, img_referencia, model_name='Facenet', distance_metric='euclidean_l2', enforce_detection=False)
            distancia_facial = result["distance"]

            # Calcular la similitud total basada en la diferencia de edad, género y la distancia facial
            similitud = peso_edad * diff_edad + peso_genero * diff_genero + distancia_facial

            # Actualizar si encontramos una similitud menor
            with result_lock:
                if similitud < distancia_minima:
                    distancia_minima = similitud
                    persona_mas_parecida = os.path.splitext(archivo_referencia)[0]  # Eliminar la extensión para obtener la categoría

            # Incrementar el índice
            index_referencia += 1

        # Reiniciar el índice para la próxima iteración
        index_referencia = 0
        # Esperar 5 segundos antes de realizar la próxima comparación
        time.sleep(5)

# Inicia los hilos
camera_thread = threading.Thread(target=read_camera)
camera_thread.daemon = True
camera_thread.start()

process_thread = threading.Thread(target=process_image)
process_thread.daemon = True
process_thread.start()

# Espera a que los hilos se inicien completamente
time.sleep(2)

# Espera a que se presione 'q' para salir del programa
while True:
    if cv2.waitKey(1) & 0xFF == ord('q'):
        exit_event.set()  # Establecer el evento para notificar a los hilos que deben finalizar
        break

# Espera a que los hilos finalicen
camera_thread.join()
process_thread.join()

# Liberar la captura de la cámara y cerrar todas las ventanas
cap.release()
cv2.destroyAllWindows()

# Eliminar la imagen temporal al finalizar
os.remove("imagen_temporal.jpg")

Action: emotion: 100%|██████████| 3/3 [00:00<00:00,  5.80it/s]
